### <ins> Predicciones del Modelo V1: CNN con Binary Cross-Entropy loss function </ins>

**1. Importar librerías, definimos las funciones de as métricas y modelos:**

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import tensorflow as tf
from keras.models import load_model
from keras import backend as K


# 1. Define la semilla
SEED = 42  # Según ChatGPT es la mejor
 
# 2. Python built-in random
random.seed(SEED)

# 3. NumPy
np.random.seed(SEED)

# 4. TensorFlow
tf.random.set_seed(SEED)

# (Opcional) Para TensorFlow más determinismo en operaciones GPU:
os.environ['TF_DETERMINISTIC_OPS'] = '1'


def recall(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    tp = K.sum(y_true * y_pred_pos)
    possible_positives = K.sum(y_true)
    return tp / (possible_positives + K.epsilon())

def precision(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    tp = K.sum(y_true * y_pred_pos)
    predicted_positives = K.sum(y_pred_pos)
    return tp / (predicted_positives + K.epsilon())

def f1_score(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * (prec * rec) / (prec + rec + K.epsilon())

def specificity(y_true, y_pred):
    y_pred_neg = 1 - K.round(K.clip(y_pred, 0, 1))
    y_true_neg = 1 - y_true
    tn = K.sum(y_true_neg * y_pred_neg)
    possible_negatives = K.sum(y_true_neg)
    return tn / (possible_negatives + K.epsilon())

def hamming_loss(y_true, y_pred):
    mismatches = K.not_equal(K.round(K.clip(y_pred, 0, 1)), y_true)
    return K.mean(K.cast(mismatches, K.floatx()))


In [3]:
modelos_delta = {}

# Iteramos de 1 a 2 porque solo tenemos un modelo delta entrenado
for i in range(1, 2):

    # Formateamos el string del path según el valor de i
    path = f"../../Modelo GoL/Modelos/modelo_delta_{i}_FBCE"

    # Cargamos los modelos y lo almacenamos en el diccionario usando una clave dinámica
    modelos_delta[f"modelo_delta_{i}_FBCE"] = load_model(
        path, 
        compile=False, 
        custom_objects={
            'accuracy': tf.keras.metrics.BinaryAccuracy(name='accuracy'),
            'recall': recall,
            'precision': precision,
            'f1_score': f1_score,
            'specificity': specificity,
            'hamming_loss': hamming_loss
        }
    )

# Buscamos los paths a todos los excels relevantes:
path = "../../Datos"
datos = []
for dirnames,_,filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('.xlsx'):
            datos.append(os.path.join(dirnames,filename))

# Cargamos los datos del test:

test = pd.read_excel((datos[0]), sheet_name = 'test', header = 0)
stops = [f'stop.{i}' for i in range(1,401)]

print(datos)

['../../Datos\\test.xlsx', '../../Datos\\train.xlsx']


2. **Jugamos al GoL con las predicciones:**

In [4]:
archivo_excel = "predicciones_FBCE.xlsx"
with pd.ExcelWriter(archivo_excel, engine='openpyxl') as writer:
    for delta in range(1, 2):
        # Llamo al modelo asociado a esta delta:
        key = f"modelo_delta_{delta}_FBCE"
        modelo_i = modelos_delta[key]
        
        # Filto los datos del test asociados a esta delta:
        delta_i = test[test['delta'] == delta]
        finales_i = np.reshape(delta_i[stops].values, (-1,20,20,1), order='F')

        # Hacemos las prediciones y las organizo como un vector fila.
        predicciones_finales, predicciones_iniciales = modelo_i.predict(finales_i)   
        
        pred_init_flat = predicciones_iniciales.reshape(len(predicciones_iniciales), -1).round(0).astype('uint8')
        pred_final_flat = predicciones_finales.reshape(len(predicciones_finales), -1).round(0).astype('uint8')

        resultados = np.concatenate([pred_final_flat, pred_init_flat], axis=1) # Final + Inicial

        columnas = [f"stop_{i}" for i in range(400)] + [f"start_{i}" for i in range(400)] 

        df_predicciones = pd.DataFrame(resultados, columns=columnas)
        
        # # Define el nombre de la hoja. En este ejemplo se usa "Iteracion_1", "Iteracion_2", etc.
        hoja = f"delta_{delta}"
        
        # # Guarda el DataFrame en la hoja actual del archivo Excel
        df_predicciones.to_excel(writer, sheet_name=hoja, index=False)

312/312 [==============================] - 29s 72ms/step
